In [ ]:
import kagglehub

path = kagglehub.dataset_download("paramaggarwal/fashion-product-images-small")
print("Dataset path:", path)


Using Colab cache for faster access to the 'fashion-product-images-small' dataset.
Dataset path: /kaggle/input/fashion-product-images-small


In [ ]:
import os

def find_file(root, filename):
    for r, d, f in os.walk(root):
        if filename in f:
            return os.path.join(r, filename)
    return None

styles_csv = find_file(path, "styles.csv")

images_dir = None
for r, d, f in os.walk(path):
    if os.path.basename(r).lower() == "images":
        images_dir = r
        break

print("styles.csv:", styles_csv)
print("images_dir:", images_dir)

assert styles_csv is not None, "styles.csv not found"
assert images_dir is not None, "images folder not found"


styles.csv: /kaggle/input/fashion-product-images-small/styles.csv
images_dir: /kaggle/input/fashion-product-images-small/myntradataset/images


In [ ]:
import pandas as pd

df = pd.read_csv(
    styles_csv,
    engine="python",
    on_bad_lines="skip"
)

print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())
df.head()


Loaded rows: 44424
Columns: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName']


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [ ]:
import numpy as np

df = df.dropna(subset=["id", "gender", "baseColour", "articleType"]).copy()

df["id"] = pd.to_numeric(df["id"], errors="coerce")
df = df.dropna(subset=["id"]).copy()
df["id"] = df["id"].astype(int)

df["filepath"] = df["id"].apply(lambda x: os.path.join(images_dir, f"{x}.jpg"))
df = df[df["filepath"].apply(os.path.exists)].copy()

df = df.sample(frac=1.0, random_state=42).reset_index(drop=True)

print("Usable rows:", len(df))
df[["id","gender","baseColour","articleType","filepath"]].head()


Usable rows: 44404


,id,gender,baseColour,articleType,filepath
0,45362,Women,Olive,Heels,/kaggle/input/fashion-product-images-small/myn...
1,22849,Men,Khaki,Casual Shoes,/kaggle/input/fashion-product-images-small/myn...
2,22372,Men,Red,Shirts,/kaggle/input/fashion-product-images-small/myn...
3,10168,Men,Black,Casual Shoes,/kaggle/input/fashion-product-images-small/myn...
4,54488,Women,Green,Handbags,/kaggle/input/fashion-product-images-small/myn...


In [ ]:
TOP_GENDER = 3
TOP_COLOR  = 8
TOP_ART    = 12

top_gender = df["gender"].value_counts().head(TOP_GENDER).index.tolist()
top_color  = df["baseColour"].value_counts().head(TOP_COLOR).index.tolist()
top_art    = df["articleType"].value_counts().head(TOP_ART).index.tolist()

df = df[
    df["gender"].isin(top_gender) &
    df["baseColour"].isin(top_color) &
    df["articleType"].isin(top_art)
].copy().reset_index(drop=True)

print("Filtered rows:", len(df))
print("Genders:", top_gender)
print("Colors:", top_color)
print("Article types:", top_art)


Filtered rows: 20147
Genders: ['Men', 'Women', 'Unisex']
Colors: ['Black', 'White', 'Blue', 'Brown', 'Grey', 'Red', 'Green', 'Pink']
Article types: ['Tshirts', 'Shirts', 'Casual Shoes', 'Watches', 'Sports Shoes', 'Handbags', 'Kurtas', 'Tops', 'Heels', 'Sunglasses', 'Wallets', 'Flip Flops']


In [ ]:
label_vocab = []
label_vocab += [f"gender:{g}" for g in top_gender]
label_vocab += [f"color:{c}" for c in top_color]
label_vocab += [f"article:{a}" for a in top_art]

label_to_idx = {l:i for i,l in enumerate(label_vocab)}
num_labels = len(label_vocab)

print("num_labels:", num_labels)
print("Example labels:", label_vocab[:10])

def make_multihot(row):
    y = np.zeros((num_labels,), dtype=np.float32)
    y[label_to_idx[f"gender:{row['gender']}"]] = 1.0
    y[label_to_idx[f"color:{row['baseColour']}"]] = 1.0
    y[label_to_idx[f"article:{row['articleType']}"]] = 1.0
    return y

X = df["filepath"].to_numpy()
Y = np.stack(df.apply(make_multihot, axis=1).to_numpy())

print("X shape:", X.shape)
print("Y shape:", Y.shape)
print("Sample multi-hot sum (should be 3):", Y[0].sum())


num_labels: 23
Example labels: ['gender:Men', 'gender:Women', 'gender:Unisex', 'color:Black', 'color:White', 'color:Blue', 'color:Brown', 'color:Grey', 'color:Red', 'color:Green']
X shape: (20147,)
Y shape: (20147, 23)
Sample multi-hot sum (should be 3): 3.0


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

np.random.seed(42)
tf.random.set_seed(42)

IMG_SIZE = (224, 224)
BATCH = 32

val_frac = 0.2
n = len(X)
n_val = int(n * val_frac)

X_train, Y_train = X[n_val:], Y[n_val:]
X_val,   Y_val   = X[:n_val],  Y[:n_val]

print("Train:", len(X_train), "Val:", len(X_val))

def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    return img, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)) \
    .shuffle(2000, seed=42) \
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)) \
    .map(load_image, num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH) \
    .prefetch(tf.data.AUTOTUNE)


Train: 16118 Val: 4029


In [ ]:
data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
])

base = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)
base.trainable = False

inputs = keras.Input(shape=(224,224,3))
x = data_aug(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_labels, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(threshold=0.5)]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 23)             │        29,463 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,287,447 (8.73 MB)

 Trainable params: 29,463 (115.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
cb = [
    keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=2, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=5, callbacks=cb)


Epoch 1/5
504/504 ━━━━━━━━━━━━━━━━━━━━ 62s 103ms/step - binary_accuracy: 0.9093 - loss: 0.2296 - val_binary_accuracy: 0.9502 - val_loss: 0.1308 - learning_rate: 0.0010
Epoch 2/5
504/504 ━━━━━━━━━━━━━━━━━━━━ 35s 68ms/step - binary_accuracy: 0.9486 - loss: 0.1333 - val_binary_accuracy: 0.9534 - val_loss: 0.1205 - learning_rate: 0.0010
Epoch 3/5
504/504 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - binary_accuracy: 0.9527 - loss: 0.1230 - val_binary_accuracy: 0.9548 - val_loss: 0.1176 - learning_rate: 0.0010
Epoch 4/5
504/504 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - binary_accuracy: 0.9533 - loss: 0.1205 - val_binary_accuracy: 0.9557 - val_loss: 0.1141 - learning_rate: 0.0010
Epoch 5/5
504/504 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - binary_accuracy: 0.9539 - loss: 0.1182 - val_binary_accuracy: 0.9568 - val_loss: 0.1123 - learning_rate: 0.0010


In [ ]:
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy(threshold=0.5)]
)

history_ft = model.fit(train_ds, validation_data=val_ds, epochs=3, callbacks=cb)


Epoch 1/3
504/504 ━━━━━━━━━━━━━━━━━━━━ 59s 95ms/step - binary_accuracy: 0.9405 - loss: 0.1530 - val_binary_accuracy: 0.9526 - val_loss: 0.1212 - learning_rate: 1.0000e-05
Epoch 2/3
504/504 ━━━━━━━━━━━━━━━━━━━━ 45s 89ms/step - binary_accuracy: 0.9525 - loss: 0.1230 - val_binary_accuracy: 0.9571 - val_loss: 0.1124 - learning_rate: 1.0000e-05
Epoch 3/3
504/504 ━━━━━━━━━━━━━━━━━━━━ 45s 89ms/step - binary_accuracy: 0.9554 - loss: 0.1155 - val_binary_accuracy: 0.9601 - val_loss: 0.1050 - learning_rate: 1.0000e-05


In [ ]:
import json, os
from google.colab import files

out_dir = "/content/task4_4_export"
os.makedirs(out_dir, exist_ok=True)

model_path = os.path.join(out_dir, "best_model_multilabel.keras")
model.save(model_path)

with open(os.path.join(out_dir, "label_vocab.json"), "w") as f:
    json.dump(label_vocab, f)

with open(os.path.join(out_dir, "label_groups.json"), "w") as f:
    json.dump({
        "gender": [l for l in label_vocab if l.startswith("gender:")],
        "color": [l for l in label_vocab if l.startswith("color:")],
        "article": [l for l in label_vocab if l.startswith("article:")],
    }, f)

print("Exported to:", out_dir)
print(os.listdir(out_dir))

files.download(model_path)
files.download(os.path.join(out_dir, "label_vocab.json"))
files.download(os.path.join(out_dir, "label_groups.json"))


Exported to: /content/task4_4_export
['label_groups.json', 'label_vocab.json', 'best_model_multilabel.keras']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>